# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import requests
import json

from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# workspace setup
ws = Workspace.from_config()
print(ws)

# dataset key info
key = 'nba_3pt_data'
data_desc = 'NBA game data from 2015 to 2023 containing 3 point percentages.'

# checking if dataset is already registered
if key in ws.datasets.keys():
    nba_dataset = ws.datasets[key]
else:
    # get data from url
    nba_data_url = 'https://raw.githubusercontent.com/mattwatson50/udacity_azure_ml_final/main/data/nba_3pt_data.csv'
    nba_dataset = Dataset.Tabular.from_delimited_files(nba_data_url)        
    # register dataset
    nba_dataset = nba_dataset.register(workspace=ws,
                               name=key,
                               description=data_desc)

# getting nba dataset
nba_df = nba_dataset.to_pandas_dataframe()
print(nba_df.head())

# choose a name for experiment
experiment_name = 'automl_nba_3pt'

experiment=Experiment(ws, experiment_name)

2024-04-08:13:55:35,834 INFO     [workspace.py:291] Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook257202/code/config.json


Workspace.create(name='quick-starts-ws-257202', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-257202')


2024-04-08:13:55:38,445 INFO     [clientbase.py:192] Created a worker pool for first use


   season_id matchup_home     season_type  min         team_name_home  \
0      22015  ATL vs. DET  Regular Season  240          Atlanta Hawks   
1      22015  GSW vs. NOP  Regular Season  240  Golden State Warriors   
2      22015  CHI vs. CLE  Regular Season  240          Chicago Bulls   
3      22015  BOS vs. PHI  Regular Season  240         Boston Celtics   
4      22015  MIA vs. CHA  Regular Season  240             Miami Heat   

   fg3m_home  fg3a_home  fg3_pct_home wl_home        team_name_away  \
0          8         27          0.30       L       Detroit Pistons   
1          9         30          0.30       W  New Orleans Pelicans   
2          7         19          0.37       W   Cleveland Cavaliers   
3          8         24          0.33       W    Philadelphia 76ers   
4         12         20          0.60       W     Charlotte Hornets   

   fg3m_away  fg3a_away  fg3_pct_away wl_away  highest_3pp highest_3pp_wl  
0         12         29          0.41       W         0.41

In [5]:
# creating a compute cluster
cluster_name = "proj-compute-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster already created.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=6)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# automl settings
automl_settings = {
    'experiment_timeout_minutes' : 30,
    'task' : 'classification',
    'primary_metric' : 'accuracy',
    'training_data' : nba_dataset,
    'label_column_name' : 'highest_3pp_wl',
    'n_cross_validations' : 5,
    'enable_early_stopping' : True,
    'enable_onnx_compatible_models' : True
}

# setting automl_config
automl_config = AutoMLConfig(compute_target=cluster_name,
                             **automl_settings)

In [7]:
# submitting experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_nba_3pt,AutoML_03172180-f10d-4b73-b3fd-b5bfdb197d81,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
# showing RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# getting best model and saving it out with onnx
automl_run, automl_best_model = remote_run.get_output(return_onnx_model=True)
print('Run ID: ' + automl_run.id)
print('Model Name: ' + automl_run.properties['model_name'])
print('Model Details:')
print(automl_best_model)

Run ID: AutoML_03172180-f10d-4b73-b3fd-b5bfdb197d81_34
Model Name: AutoML03172180f34
Model Details:


In [10]:
# convert and save the model
OnnxConverter.save_onnx_model(automl_best_model, file_path="./automl_best_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
# register the model
model_name = 'nba_3pt_voting_ensemble'
model_path = 'outputs/model.pkl'
model_description = 'Voting Ensemble Model for NBA 3PT Percentage Win Analysis'
registered_model = automl_run.register_model(model_name=model_name, model_path=model_path, description=model_description)
model = Model(ws, 'nba_3pt_voting_ensemble')

# creating inference config and deploying the model as a web service
service_name = 'nba-data-env-service'

# get the scoring file directly from azure by downloading it through the GUI
inference_config = InferenceConfig(entry_script='./scoring_file_v_1_0_0.py')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-04-08 16:11:21+00:00 Creating Container Registry if not exists.
2024-04-08 16:11:21+00:00 Use the existing image.
2024-04-08 16:11:22+00:00 Submitting deployment to compute..
2024-04-08 16:11:27+00:00 Checking the status of deployment nba-data-env-service..
2024-04-08 16:15:03+00:00 Checking the status of inference endpoint nba-data-env-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(service.get_logs())

2024-04-08T15:52:05,614619383+00:00 - rsyslog/run 
2024-04-08T15:52:05,629135506+00:00 - gunicorn/run 
2024-04-08T15:52:05,635584849+00:00 | gunicorn/run | 
2024-04-08T15:52:05,648643439+00:00 | gunicorn/run | ###############################################
2024-04-08T15:52:05,649494558+00:00 - nginx/run 
2024-04-08T15:52:05,650489480+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-04-08T15:52:05,655070282+00:00 | gunicorn/run | ###############################################
2024-04-08T15:52:05,660335299+00:00 | gunicorn/run | 
2024-04-08T15:52:05,671114938+00:00 | gunicorn/run | 
2024-04-08T15:52:05,678991313+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-04-08T15:52:05,682928701+00:00 | gunicorn/run | 
2024-04-08T15:52:05,684394733+00:00 | gunicorn/run | 
2024-04-08T15:52:05,689147239+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
# get the scoring uri and the auth key from the endpoint in azure under the consume section
scoring_uri = "http://4ab0e724-1492-41f2-b465-b937a5d655f0.westeurope.azurecontainer.io/score"
key = "CqGmVyg0DAfAboCglzixP3GQpueh26IF"

# sample data, ensure this is formatted correctly if you wish to add more
data = {"data":
           [
              {
                'season_id': 22015,
                'matchup_home': 'ATL vs. DET',
                'season_type': 'Regular Season',
                'min': 240,
                'team_name_home': 'Atlanta Hawks',
                'fg3m_home': 8,
                'fg3a_home': 27,
                'fg3_pct_home': 0.296,
                'wl_home': 'L',
                'team_name_away': 'Detroit Pistons',
                'fg3m_away': 12,
                'fg3a_away': 29,
                'fg3_pct_away': 0.414,
                'wl_away': 'W',
                'highest_3pp': 0.414
      },
   ]
}
# convert to json and save out file
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# setting context type to json
headers = {"Content-Type": "application/json"}
# passing auth
headers["Authorization"] = f"Bearer {key}"

# make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": ["W"]}


TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
# getting and printing all logs
endpoint_logs = service.get_logs()
print(endpoint_logs)

# deleting the service
service.delete()

2024-04-08T16:13:42,183336066+00:00 - rsyslog/run 
2024-04-08T16:13:42,189992968+00:00 - gunicorn/run 
2024-04-08T16:13:42,194171032+00:00 | gunicorn/run | 
2024-04-08T16:13:42,196737971+00:00 | gunicorn/run | ###############################################
2024-04-08T16:13:42,201813249+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-04-08T16:13:42,203269171+00:00 - nginx/run 
2024-04-08T16:13:42,203546375+00:00 | gunicorn/run | ###############################################
2024-04-08T16:13:42,204853795+00:00 | gunicorn/run | 
2024-04-08T16:13:42,212646614+00:00 | gunicorn/run | 
2024-04-08T16:13:42,223146975+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-04-08T16:13:42,226321624+00:00 | gunicorn/run | 
2024-04-08T16:13:42,230717291+00:00 | gunicorn/run | 
2024-04-08T16:13:42,234593850+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
